In [1]:
import numpy as np

In [2]:
import apple

In [3]:
train_images=np.load('data/train-images.npy')

In [4]:
reshape=apple.jit("[♭`{3∘[2,3,4]} (x::Arr (60000 × 28 × 28 × 1) float)]")

In [5]:
train_images_v=reshape(train_images)

In [6]:
train_labels=np.load('data/train-labels.npy')

In [7]:
def sigmoid(x):
    return 1/(np.exp(-x)+1)

In [8]:
def d_sigmoid(x):
    return (np.exp(-x))/((np.exp(-x)+1)**2)

In [9]:
def softmax(x):
    exp_element=np.exp(x-x.max())
    return exp_element/np.sum(exp_element,axis=0)

In [10]:
def d_softmax(x):
    exp_element=np.exp(x-x.max())
    return exp_element/np.sum(exp_element,axis=0)*(1-exp_element/np.sum(exp_element,axis=0))

In [11]:
def init(x,y):
    return np.random.uniform(-1.,1.,(x,y))/np.sqrt(x*y)

In [12]:
np.random.seed(17)

In [13]:
l1=init(28*28,128)

In [14]:
l2=init(128,10)

In [15]:
vmap=apple.jit("((λn.[?x=n,.1::float,.0]'irange 0 9 1)')")

In [16]:
train_labels_v=vmap(train_labels)

In [17]:
def train_py(x,y,l1,l2):
    targets = np.zeros((len(y),10), np.float64)
    targets[range(targets.shape[0]),y] = 1
    
    x_l1p=x.dot(l1)
    x_sigmoid=sigmoid(x_l1p)
    x_l2p=x_sigmoid.dot(l2)
    out=softmax(x_l2p)
        
    error=2*(out-targets)/out.shape[0]*d_softmax(x_l2p)
    update_l2=x_sigmoid.T@error
    
    error=((l2).dot(error.T)).T*d_sigmoid(x_l1p)
    update_l1=x.T@error
    
    return l1+update_l1,l2+update_l2

In [18]:
train=apple.jit('''
λx.λtargets.
λl1.λl2.
  {
    xl1p ⟜ [ℯ(_x)]`{0}(x%.l1);
    -- fw
    xSigmoid ⟜ [1%(1+x)]`{0} xl1p;
    xl2p ⟜ xSigmoid%.l2;
    a ⟜ {m ⟜ (⋉)/* _1 xl2p; [e:(x-m)]`{0} xl2p};
    out ⟜ ⍉((λxs. {s⟜ (+)/xs; (%s)'xs})`{1} (a::M float));
    dsoftmax_l2 ← [x*(1-x)]`{0} out;
    -- bw
    error ⟜ (*)`{0,0} ({n⟜2%(ℝ(𝓉out)); [x*n]`{0} ((-)`{0,0} out targets)}) dsoftmax_l2;
    ul2 ← (⍉xSigmoid)%.error;
    ul1 ← (⍉x)%.((*)`{0,0} (⍉(l2%.(⍉error))) ([x%(1+x)^2]`{0} xl1p));
    ((+)`{0,0} l1 ul1, (+)`{0,0} l2 ul2)
  }
''')

In [19]:
train(train_images_v,train_labels_v,l1,l2)

(array([[-1.29637294e-03,  1.93108056e-04, -1.94756914e-03, ...,
          2.45069678e-03, -1.96610909e-03,  1.78056911e-03],
        [ 9.08670865e-04, -9.21828209e-05,  2.70875527e-03, ...,
          2.26569033e-03, -1.03762444e-03, -2.30342918e-03],
        [ 6.92568035e-04,  1.01154124e-03,  3.01171972e-03, ...,
          2.75982754e-03, -2.97280076e-04,  2.61869556e-03],
        ...,
        [-2.27470948e-03,  3.89819298e-04, -1.68065782e-03, ...,
         -6.10246153e-04, -2.07402878e-03,  1.66304741e-03],
        [ 1.04147115e-03, -1.64259621e-03, -2.77502269e-03, ...,
         -3.57187593e-04,  2.24773693e-03,  2.60555625e-03],
        [-2.76720736e-03, -8.55114367e-04, -1.95089616e-03, ...,
         -7.15024703e-04,  1.54345339e-04,  3.68339989e-04]],
       shape=(784, 128)),
 array([[ 0.0213697 , -0.0126215 , -0.0269387 , ...,  0.00998378,
         -0.00559339, -0.00853609],
        [ 0.00404237, -0.00586295,  0.00138553, ..., -0.01798783,
         -0.00901585,  0.01142074],


In [20]:
train_py(train_images_v,train_labels,l1,l2)

(array([[-1.29637294e-03,  1.93108056e-04, -1.94756914e-03, ...,
          2.45069678e-03, -1.96610909e-03,  1.78056911e-03],
        [ 9.08670865e-04, -9.21828209e-05,  2.70875527e-03, ...,
          2.26569033e-03, -1.03762444e-03, -2.30342918e-03],
        [ 6.92568035e-04,  1.01154124e-03,  3.01171972e-03, ...,
          2.75982754e-03, -2.97280076e-04,  2.61869556e-03],
        ...,
        [-2.27470948e-03,  3.89819298e-04, -1.68065782e-03, ...,
         -6.10246153e-04, -2.07402878e-03,  1.66304741e-03],
        [ 1.04147115e-03, -1.64259621e-03, -2.77502269e-03, ...,
         -3.57187593e-04,  2.24773693e-03,  2.60555625e-03],
        [-2.76720736e-03, -8.55114367e-04, -1.95089616e-03, ...,
         -7.15024703e-04,  1.54345339e-04,  3.68339989e-04]],
       shape=(784, 128)),
 array([[ 0.0213697 , -0.0126215 , -0.0269387 , ...,  0.00998378,
         -0.00559339, -0.00853609],
        [ 0.00404237, -0.00586295,  0.00138553, ..., -0.01798783,
         -0.00901585,  0.01142074],


In [21]:
%timeit train(train_images_v,train_labels_v,l1,l2)

2.38 s ± 67.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
%timeit train_py(train_images_v,train_labels,l1,l2)

389 ms ± 6.38 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
